In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
import nnqs
import sampler

2025-07-09 16:29:59.532751: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752071399.554839   39375 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752071399.561338   39375 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752071399.578000   39375 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752071399.578019   39375 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752071399.578022   39375 computation_placer.cc:177] computation placer alr

In [2]:
n_spins = 16
# density = hidden/visible
density = 4
n_hidden_spins = int(n_spins * density)

wave = nnqs.RBM(n_visible=n_spins, n_hidden=n_hidden_spins)
mrt2_sampler = sampler.MRT2(n_sites=n_spins, batch_size=10, n_chains=1, n_sweeps=10)

print(mrt2_sampler.current_state)

<tf.Variable 'mrt2_state:0' shape=(1, 16) dtype=int32, numpy=array([[1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1]], dtype=int32)>


I0000 00:00:1752071402.868954   39375 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2609 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5


In [3]:
def target_log_prob(*state_parts):
    print("State parts:", state_parts)
    if len(state_parts) == 1:
        state = state_parts[0]
    else:
        state = state_parts[0]
    
    return wave.log_prob(state)


metro_kernel = tfp.mcmc.RandomWalkMetropolis(
    target_log_prob_fn=target_log_prob,
    new_state_fn=mrt2_sampler._single_site_flip
)

init_state = metro_kernel.bootstrap_results(mrt2_sampler.current_state)

print("Initial state:", init_state)

result = metro_kernel.one_step(
    current_state=mrt2_sampler.current_state,
    previous_kernel_results=init_state,
    seed=42
)

print("New state after one step:", result[0])
print("Kernel results:", result[1])


State parts: (<tf.Tensor: shape=(1, 16), dtype=int32, numpy=array([[1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1]], dtype=int32)>,)
Initial state: MetropolisHastingsKernelResults(
  accepted_results=UncalibratedRandomWalkResults(
      log_acceptance_correction=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
      target_log_prob=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([89.29968], dtype=float32)>,
      seed=[]
    ),
  is_accepted=<tf.Tensor: shape=(1,), dtype=bool, numpy=array([ True])>,
  log_accept_ratio=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
  proposed_state=<tf.Variable 'mrt2_state:0' shape=(1, 16) dtype=int32, numpy=array([[1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1]], dtype=int32)>,
  proposed_results=UncalibratedRandomWalkResults(
      log_acceptance_correction=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
      target_log_prob=<tf.Tensor: shape=(1,), dtype=float32, numpy

In [4]:
samples = mrt2_sampler.sample_with_diagnostics(wave)

print("Generated samples:")
print(samples)


Generated samples:
(<tf.Tensor: shape=(10, 16), dtype=float32, numpy=
array([[1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.]],
      dtype=float32)>, <tf.Tensor: shape=(), dtype=float32, numpy=0.699999988079071>, <tf.Tensor: shape=(10, 1), dtype=bool, numpy=
array([[ True],
       [ True],
       [False],
       [False],
   

In [5]:
print(mrt2_sampler.current_state)
print(wave.a)
print(wave.b)
print(wave.W)

wave.log_psi(mrt2_sampler.current_state)

<tf.Variable 'mrt2_state:0' shape=(1, 16) dtype=int32, numpy=array([[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]], dtype=int32)>
<tf.Variable 'visible_bias:0' shape=(16,) dtype=float32, numpy=
array([-0.00661019, -0.16981201,  0.09483193, -0.06008063, -0.09186407,
       -0.04272849, -0.00348994,  0.03309365, -0.03137584, -0.01620407,
       -0.07359698, -0.02693218,  0.08760764,  0.07020556, -0.05304328,
       -0.15884362], dtype=float32)>
<tf.Variable 'hidden_bias:0' shape=(64,) dtype=float32, numpy=
array([-0.03923926,  0.03368663, -0.2347214 ,  0.07666726,  0.03409345,
       -0.05664311,  0.11389887, -0.09986163, -0.03448831, -0.3251216 ,
       -0.14559953, -0.01346879,  0.02683028, -0.03236302,  0.11542135,
       -0.06658442,  0.03160604,  0.00428767, -0.05838616, -0.00056354,
        0.08442327, -0.05951727,  0.20788567, -0.02442737, -0.04564942,
       -0.02173257,  0.11523371,  0.07958623, -0.01662771,  0.08083832,
        0.00124801,  0.07552718, -0.09575661,  0.008638

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([44.358906], dtype=float32)>

In [6]:
wave_test = nnqs.RBM(n_visible=8, n_hidden=8)

states0 = tf.zeros((1, 8), dtype=tf.float32)  # All zeros
states1 = tf.ones((1, 8), dtype=tf.float32)   # All ones
states_r = tf.random.uniform((1, 8), maxval=2, dtype=tf.int32)
states_r = tf.cast(states_r, tf.float32)

print("Log prob (all zeros):", wave_test.log_prob(states0))
print("Log prob (all ones):", wave_test.log_prob(states1))  
print("Log prob (random):", wave_test.log_prob(states_r))
print("Prob (all zeros):", tf.exp(wave_test.log_prob(states0)))
print("Prob (all ones):", tf.exp(wave_test.log_prob(states1)))
print("Prob (random):", tf.exp(wave_test.log_prob(states_r)))

Log prob (all zeros): tf.Tensor([12.392055], shape=(1,), dtype=float32)
Log prob (all ones): tf.Tensor([10.46623], shape=(1,), dtype=float32)
Log prob (random): tf.Tensor([11.93464], shape=(1,), dtype=float32)
Prob (all zeros): tf.Tensor([240880.11], shape=(1,), dtype=float32)
Prob (all ones): tf.Tensor([35109.617], shape=(1,), dtype=float32)
Prob (random): tf.Tensor([152457.31], shape=(1,), dtype=float32)


In [7]:
test_states = tf.random.uniform((5, 8), maxval=2, dtype=tf.int32)
print("Test states:", test_states.numpy())
test_states = tf.cast(test_states, tf.float32)
print("Test states (as float):", test_states.numpy())
log_probs = wave_test.log_prob(test_states)
sum_visible = tf.reduce_sum(wave_test.a * test_states, axis=1)
sum_hidden = tf.reduce_sum(tf.math.log(2.0*(tf.math.cosh(wave_test.b + tf.matmul(test_states, wave_test.W)))), axis=1)
print("Sum of visible biases:", sum_visible.numpy())
print("Sum of hidden biases:", sum_hidden.numpy())
log_prob_test = sum_visible + sum_hidden
print("Log prob (manual):", log_prob_test.numpy())
print("Prob:", tf.exp(log_prob_test).numpy())

print("Log prob:", log_probs)
print("Prob:", tf.exp(log_probs))

Test states: [[0 1 0 0 0 1 0 0]
 [0 1 0 1 1 1 0 1]
 [1 0 1 0 0 1 1 1]
 [0 1 1 0 0 0 0 0]
 [1 1 0 1 1 0 1 0]]
Test states (as float): [[0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 1. 1. 0. 1.]
 [1. 0. 1. 0. 0. 1. 1. 1.]
 [0. 1. 1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 1. 0. 1. 0.]]
Sum of visible biases: [-0.06577981 -0.15950862  0.02992091  0.05914544 -0.09354654]
Sum of hidden biases: [5.615298  5.585548  5.588237  5.648363  5.6377125]
Log prob (manual): [5.549518  5.426039  5.618158  5.7075086 5.544166 ]
Prob: [257.11362 227.24739 275.38162 301.1199  255.74123]
Log prob: tf.Tensor([10.683421 11.188381 11.93464  10.028568 11.463413], shape=(5,), dtype=float32)
Prob: tf.Tensor([ 43626.547  72285.67  152457.31   22664.799  95169.36 ], shape=(5,), dtype=float32)


In [8]:
# test_state = tf.constant([[1, 0, 1, 0, 1, 0, 1, 0]], dtype=tf.int32)
# flipped = mrt2_sampler._single_site_flip(test_state)
# print("Original:", test_state)
# print("Log prob (Original)", wave_test.log_prob(test_state))
# print("Flipped: ", flipped)
# print("Log prob (Flipped)", wave_test.log_prob(flipped))
# print("Ratio of log:", wave_test.log_prob(flipped) - wave_test.log_prob(test_state))

In [9]:
wave.trainable_variables

(<tf.Variable 'weights:0' shape=(16, 64) dtype=float32, numpy=
 array([[ 0.0789635 , -0.09652181, -0.04346514, ..., -0.05647821,
          0.11199776, -0.17064828],
        [-0.05283898,  0.08643264, -0.13366576, ..., -0.10619555,
         -0.08226819, -0.2714565 ],
        [-0.1007392 ,  0.03295675,  0.09590537, ..., -0.04913757,
          0.04265376,  0.01807496],
        ...,
        [-0.17030925, -0.1203291 , -0.00862198, ...,  0.07484175,
          0.08767328, -0.26947027],
        [ 0.1125889 , -0.09822967, -0.18798983, ...,  0.16081111,
         -0.04793103, -0.03701479],
        [-0.02614751,  0.0162499 ,  0.0460693 , ...,  0.10870371,
         -0.00344296,  0.01723055]], dtype=float32)>,
 <tf.Variable 'visible_bias:0' shape=(16,) dtype=float32, numpy=
 array([-0.00661019, -0.16981201,  0.09483193, -0.06008063, -0.09186407,
        -0.04272849, -0.00348994,  0.03309365, -0.03137584, -0.01620407,
        -0.07359698, -0.02693218,  0.08760764,  0.07020556, -0.05304328,
        -0

In [10]:
import hamiltonian

In [11]:
l_lattice = 4
n_spins = l_lattice * l_lattice
density = 4
n_hidden_spins = int(n_spins * density)

wave = nnqs.RBM(n_visible=n_spins, n_hidden=n_hidden_spins)

tfih = hamiltonian.TFIH(L=l_lattice, J=1.0, h=0.0, pbc=True)

m_sampler = sampler.MRT2(n_sites=n_spins, batch_size=100, n_chains=10, n_sweeps=5)
samples = m_sampler.sample(wave)
local_energies = tfih.local_energy(samples, wave)
print("Local energies:", tf.shape(local_energies))
with tf.GradientTape() as tape:
    log_psi = wave.log_psi(samples)

grad_log_psi = tape.jacobian(log_psi, wave.trainable_variables)
#print(grad_log_psi)
#print("Gradients of log_psi:", [g.shape if g is not None else None for g in grad_log_psi])

# VMC gradient formula
mean_energy = tf.reduce_mean(local_energies)
gradients = []

for grad_i in grad_log_psi:
    if grad_i is not None:
        # Covariance term: <E_loc * grad_log_psi> - <E_loc> * <grad_log_psi>
        expand_shape = [tf.shape(local_energies)[0]] + [1] * (len(grad_i.shape) - 1)
        e_loc_expanded = tf.reshape(local_energies, expand_shape)
        loc_grad = e_loc_expanded * grad_i
        grad_energy = tf.reduce_mean(loc_grad * grad_i, axis=0)
        grad_log_psi_mean = tf.reduce_mean(grad_i, axis=0)
        
        vmc_grad = 2.0 * (grad_energy - mean_energy * grad_log_psi_mean)
        gradients.append(vmc_grad)
    else:
        gradients.append(None)
# Convert gradients to a list of tensors
gradients = [tf.convert_to_tensor(g) if g is not None else None for g in gradients]

print("VMC Gradients:", gradients)

Local energies: tf.Tensor([100], shape=(1,), dtype=int32)
VMC Gradients: [<tf.Tensor: shape=(16, 64), dtype=float32, numpy=
array([[ 0.93206596,  0.49009752,  2.306085  , ...,  1.6084576 ,
         1.9453397 , -0.26243877],
       [-5.473702  , -2.8271618 , -2.8131778 , ..., -5.020003  ,
        -3.764181  , -5.2979774 ],
       [-6.1465435 , -2.576196  , -3.2080474 , ..., -5.0712986 ,
        -3.6260624 , -5.8780117 ],
       ...,
       [ 3.8453264 ,  3.820355  ,  4.3178434 , ...,  4.082816  ,
         4.502462  ,  3.0688972 ],
       [ 6.0933676 ,  5.734306  ,  6.577508  , ...,  6.110381  ,
         6.5933824 ,  5.1042175 ],
       [ 6.0933676 ,  5.734306  ,  6.577508  , ...,  6.110381  ,
         6.5933824 ,  5.1042175 ]], dtype=float32)>, <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([ -9.631999 , -19.7456   , -20.2272   ,  -2.8896008, -13.0032   ,
        -2.8896008,   0.       , -10.5952   , -33.711998 , -20.7088   ,
         0.       ,  -5.7791996, -20.7088   ,  -3.85280

$$\nabla_\theta E = 2 \text{Re}\left[\langle E_{\text{loc}} \nabla_\theta \log \psi \rangle - \langle E_{\text{loc}} \rangle \langle \nabla_\theta \log \psi \rangle\right]$$

In [12]:
print("a:", wave.a.numpy())
print("b:", wave.b.numpy())
print("W:", wave.W.numpy())
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# Perform a single optimization step
samples = m_sampler.sample(wave)
local_energies = tfih.local_energy(samples, wave)

# Apply gradients
optimizer.apply_gradients(zip(gradients, wave.trainable_variables))

print("Updated wave function parameters:")
print("a:", wave.a.numpy())
print("b:", wave.b.numpy())
print("W:", wave.W.numpy())



a: [-0.15573046  0.03690806  0.23080447 -0.19067125 -0.05475033 -0.07543457
  0.02422273 -0.13896626  0.02317513 -0.04526901  0.17863843 -0.01124273
 -0.03497537 -0.17624156 -0.03939826 -0.02663275]
b: [-2.22570468e-02 -8.76288041e-02 -3.00029423e-02  4.19744402e-02
 -6.63370192e-02  3.61432880e-02 -3.69122289e-02  3.50778028e-02
  4.62569743e-02  2.26784982e-02  1.50829360e-01  5.17010465e-02
  1.51102945e-01 -1.04131691e-01  8.12861845e-02  1.97524205e-02
  5.64064691e-03 -4.98948945e-03  8.18362401e-04 -5.64782694e-02
 -8.63307330e-04 -5.94932139e-02 -1.03089087e-01  1.23947561e-01
  1.55869117e-02 -2.22086590e-02 -2.03373894e-01  1.36835768e-03
  2.08343100e-02  1.47717511e-02 -1.48827126e-02  3.20006697e-03
 -2.80348724e-03 -9.02484804e-02 -4.32839058e-02 -7.90492222e-02
 -1.35239912e-02  1.67230055e-01  5.90736754e-02 -2.20594816e-02
  4.57554348e-02 -5.90905547e-02 -4.81045581e-02 -7.25819468e-02
 -1.08182780e-01 -1.41272560e-01  7.35921636e-02  8.11808035e-02
  9.44317803e-02 -

In [16]:
import optimizer

l_lattice = 2
n_spins = l_lattice * l_lattice
density = 4
n_hidden_spins = int(n_spins * density)

wave = nnqs.RBM(n_visible=n_spins, n_hidden=n_hidden_spins)

tfih = hamiltonian.TFIH(L=l_lattice, J=1.0, h=0.0, pbc=True)

m_sampler = sampler.MRT2(n_sites=n_spins, batch_size=100, n_chains=10, n_sweeps=5)

opt = optimizer.VMC(
    wave_function=wave,
    hamiltonian=tfih,
    sampler=m_sampler,
    learning_rate=0.01
)

energies, variances = opt.train(
    n_iterations=10)

Iteration 0: Energy = -6.080000, Variance = 11.673601
Iteration 9: Energy = -8.000000, Variance = 0.000000


In [17]:
print(m_sampler.current_state)
print("a:", wave.a.numpy())
print("b:", wave.b.numpy())
print("W:", wave.W.numpy())

<tf.Variable 'mrt2_state:0' shape=(10, 4) dtype=int32, numpy=
array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]], dtype=int32)>
a: [ 0.02643989 -0.01603901 -0.05894557  0.08284626]
b: [-3.1313840e-01  1.5367303e-02 -8.0001131e-02  2.8864685e-02
  9.4141796e-02 -1.2401396e-01 -1.5624380e-01 -1.1739098e-01
 -2.7436051e-01 -7.3228553e-02  6.0353339e-02 -2.6123500e-01
 -7.3414296e-05 -4.4970047e-01 -2.7358425e-01 -2.3527579e-01]
W: [[-0.22808294 -0.15225148 -0.32848662 -0.21184477 -0.1136045  -0.2698256
  -0.08693966  0.01528228 -0.08714174 -0.17079496 -0.20783201 -0.03303826
  -0.15997252 -0.1477858  -0.16375345 -0.02589709]
 [-0.24239135 -0.13724422 -0.11497303 -0.13187295 -0.15085861 -0.16103064
  -0.16858688 -0.1321153  -0.21450026 -0.19195937 -0.0679836  -0.11129532
  -0.0623491  -0.00732991 -0.20457967 -0.00049532]
 [-0.0796801